<img src="img/01.png">

#### According to [REF1](../README.md) :

Like the Series object discussed in the previous section, the DataFrame can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary.

# 03. Pandas - Data Frames  - chaptel 2
## 03.01 Data Frames - concatenation, assign
* First we need to prepare DataFrame with area

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

In [2]:
# first let's read DataFrame
df_area = pd.read_csv('../92_data/usa_states_area.csv', index_col=0, header=[0,1])
df_area.index.name = "state" # setting index name as "state"
df_area.head()

Total                     Land                             Water  \
            Rank      sq_mi      km2 Rank      sq_mi      km2       %  Rank   
state                                                                         
Alaska         1  665384.04  1723337    1  570640.95  1477953  85.76%     1   
Texas          2  268596.46   695662    2  261231.71   676587  97.26%     8   
California     3  163696.32   423972    3  155779.22   403466  95.16%     6   
Montana        4  147039.71   380831    4  145545.80   376962  98.98%    26   
New Mexico     5  121590.30   314917    5  121298.15   314161  99.76%    49   

                                      
               sq_mi     km2       %  
state                                 
Alaska      94743.10  245384  14.24%  
Texas        7364.75   19075   2.74%  
California   7915.52   20501   4.84%  
Montana      1493.91    3869   1.02%  
New Mexico    292.15     757   0.24%

In [3]:
# dealing with multiindex is pretty hard
# pd.IndexSlice is the thing we are looking for!
idx = pd.IndexSlice
df_area = df_area.loc[idx[:], idx[:, 'km2']]
df_area.head()

,Total,Land,Water
,km2,km2,km2
state,,,
Alaska,1723337,1477953,245384
Texas,695662,676587,19075
California,423972,403466,20501
Montana,380831,376962,3869
New Mexico,314917,314161,757


In [4]:
# now - Let's flat the columns by passing them directly as a property
df_area.columns = ["_".join([area_type,unit]) for area_type, unit in df_area.columns]
df_area.head()

,Total_km2,Land_km2,Water_km2
state,,,
Alaska,1723337,1477953,245384
Texas,695662,676587,19075
California,423972,403466,20501
Montana,380831,376962,3869
New Mexico,314917,314161,757


* next we need to get the data with population

In [5]:
ser_pop = pd.read_csv('../92_data/usa_states_population.csv', 
                     header=None, 
                     squeeze=True, 
                     index_col=0, 
                     names=['Population'])
ser_pop.head()

Alabama        4833722
Alaska          735132
Arizona        6626624
Arkansas       2959373
California    38332521
Name: Population, dtype: int64

* Let's stack them together

In [6]:
# you can see a pandas warning below
# IMPORTANT * pd.concat is much more 
df_density = pd.concat([df_area, ser_pop], axis=1)
df_density.head()

/home/lcs123/venvs/dstip_venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,Total_km2,Land_km2,Water_km2,Population
Alabama,135767,131171,4597,4833722
Alaska,1723337,1477953,245384,735132
Arizona,295234,294207,1026,6626624
Arkansas,137732,134771,2961,2959373
California,423972,403466,20501,38332521


In [7]:
# to get rid of it please pass `sort` argument as mentioned in the warning
df_density = pd.concat([df_area, ser_pop], axis=1, sort=True)
df_density.head()

,Total_km2,Land_km2,Water_km2,Population
Alabama,135767,131171,4597,4833722
Alaska,1723337,1477953,245384,735132
Arizona,295234,294207,1026,6626624
Arkansas,137732,134771,2961,2959373
California,423972,403466,20501,38332521


* Let's add new computation. We are interested in population density.

In [8]:
# to add a column with new data is quite simple
# add a new column "Pleople per Total Area"
df_density['ppl_pT'] = df_density['Population']/df_density['Total_km2']
df_density['ppl_pL'] = df_density['Population']/df_density['Land_km2']
df_density.head()

,Total_km2,Land_km2,Water_km2,Population,ppl_pT,ppl_pL
Alabama,135767,131171,4597,4833722,35.603070,36.850539
Alaska,1723337,1477953,245384,735132,0.426575,0.497399
Arizona,295234,294207,1026,6626624,22.445328,22.523679
Arkansas,137732,134771,2961,2959373,21.486459,21.958530
California,423972,403466,20501,38332521,90.412860,95.008058


In [9]:
# *IMPORTANT*: `df_density.assign` using we create NEW dataframe
df_density = df_density.assign(ppl_pW=lambda x: x['Population']/x['Water_km2'])
df_density.head()

,Total_km2,Land_km2,Water_km2,Population,ppl_pT,ppl_pL,ppl_pW
Alabama,135767,131171,4597,4833722,35.603070,36.850539,1051.494888
Alaska,1723337,1477953,245384,735132,0.426575,0.497399,2.995843
Arizona,295234,294207,1026,6626624,22.445328,22.523679,6458.697856
Arkansas,137732,134771,2961,2959373,21.486459,21.958530,999.450523
California,423972,403466,20501,38332521,90.412860,95.008058,1869.787864


In [10]:
df_density.sort_values('ppl_pT', ascending=False)

,Total_km2,Land_km2,Water_km2,Population,ppl_pT,ppl_pL,ppl_pW
New Jersey,22591,19047,3544,8899339,393.932938,467.230482,2511.100169
Rhode Island,4001,2678,1324,1051511,262.812047,392.647872,794.192598
Connecticut,14357,12542,1816,3596080,250.475726,286.723011,1980.220264
Massachusetts,27336,20202,7134,6692824,244.835528,331.295119,938.158677
Maryland,32131,25142,6990,5928814,184.520059,235.813141,848.185122
Delaware,6446,5047,1399,925749,143.616041,183.425599,661.721944
New York,141297,122057,19240,19651127,139.076746,160.999590,1021.368347
Florida,170312,138887,31424,19552860,114.806121,140.782507,622.226960
Pennsylvania,119280,115883,3397,12773801,107.090887,110.230155,3760.318222
Ohio,116098,105829,10269,11570808,99.664146,109.334946,1126.770669


**EXCERCISE 04.04**

1. Find TOP5 states with the most relative particiaption of water in total area?

In [11]:
### YOUR CODE HERE:
pass

In [12]:
### TO SHOW THE SOLUTION USE LINE BELOW ###
# %load ../91_solutions/ex4_4.py

## 03.01 Data Frames - merge, join 
Pretty common issue when you are dealing with data is combining them with respect to the particular column. In the case below we need to assign a new column with `manager` to the DataFrame with employees.

In [13]:
df_empl = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', 'John', 'Larry'],
                        'department': ['Accounting', 'Engineering', 'Engineering', 'HR', 'Engineering', 'HR']})
df_manag = pd.DataFrame({'department_name': ['Accounting', 'Engineering', 'HR'],
                         'manager': ['Chris', "Betty", "Fred"]})
display(df_empl)
display(df_manag)

,employee,department
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,John,Engineering
5,Larry,HR


,department_name,manager
0,Accounting,Chris
1,Engineering,Betty
2,HR,Fred


One way to do this (which we already know pertty well) is using concatenation. The difficult part is to take care about proper indexing. Let's have a look...

In [14]:
df_manag_temp_idx = df_manag.set_index('department_name')
df_manag_temp_idx

,manager
department_name,
Accounting,Chris
Engineering,Betty
HR,Fred


In [15]:
df_manag_temp_column=df_manag_temp_idx.loc[df_empl['department']]
df_manag_temp_column

,manager
department_name,
Accounting,Chris
Engineering,Betty
Engineering,Betty
HR,Fred
Engineering,Betty
HR,Fred


In [16]:
# in this case we had to reset the index, otherwise dataframes won't concatenate
df_empl.assign(manager = df_manag_temp_column.reset_index(drop=True))

,employee,department,manager
0,Bob,Accounting,Chris
1,Jake,Engineering,Betty
2,Lisa,Engineering,Betty
3,Sue,HR,Fred
4,John,Engineering,Betty
5,Larry,HR,Fred


and viola...  
just kidding - this is horrible...  
Forunately there will be no need to suffer anymore. Meet your new friends `pd.merge` and `pd.DataFrame.join`!  
Now let's do this with style :)

In [17]:
# MERGE is very intuitive but it leaves two columns in the result dataframe
pd.merge(df_empl, df_manag, left_on='department', right_on='department_name', how='left')

,employee,department,department_name,manager
0,Bob,Accounting,Accounting,Chris
1,Jake,Engineering,Engineering,Betty
2,Lisa,Engineering,Engineering,Betty
3,Sue,HR,HR,Fred
4,John,Engineering,Engineering,Betty
5,Larry,HR,HR,Fred


In [18]:
# JOIN is slightly less intuitive but it gives clear result
# IMPORTANT - in `join` method we specify join key in the first dataframe (method's object) using
# `on` parameter, the second dataframe index is taken as a key by default!
df_empl.join(df_manag.set_index("department_name"), on='department')

,employee,department,manager
0,Bob,Accounting,Chris
1,Jake,Engineering,Betty
2,Lisa,Engineering,Betty
3,Sue,HR,Fred
4,John,Engineering,Betty
5,Larry,HR,Fred


**EXCERCISE 04.05**
Given the KPI's of some application for:
- different languages 
- different releases  

Please design a tool to compare releases with each other. 

IMPORTANT: this is not well defined task. Try to use your creativity! It would be really great to find some usage for the new learned functions

In [19]:
### DATA ###
features_index = ["PL", "DE", "EN", "IT", "ES", "PT", "RU", "FR", "NL", "GR"]
experiments_data = {
    "1.1": [0.0958, 0.0653, 0.0414, 0.9581, 0.4818, 0.2347, 0.6384, 0.2423, 0.8497, 0.0386],
    "1.2": [0.4689, 0.0653, 0.0414, 0.9328, 0.4818, 0.2347, 0.7263, 0.2423, 0.8497, 0.0386],
    "1.3": [0.4689, 0.0653, 0.4161, 0.9328, 0.4818, 0.7263, 0.7263, 0.2423, 0.8497, 0.0386],
    "1.4": [0.4689, 0.3330, 0.5561, 0.9328, 0.4818, 0.7263, 0.7263, 0.5629, 0.8497, 0.3462],
    "1.5": [0.4689, 0.1910, 0.5561, 0.9328, 0.7878, 0.7263, 0.7263, 0.5629, 0.8045, 0.4265],
    "2.0": [0.7571, 0.3330, 0.5561, 0.9328, 0.8651, 0.7263, 0.0458, 0.5629, 0.8045, 0.4265],
    "2.1": [0.7571, 0.3407, 0.9218, 0.9328, 0.8651, 0.7263, 0.8897, 0.5629, 0.8897, 0.4265],
}

#### SOLUTION A: using `pd.DataFrame.apply` method 


`IN`:
```python
df_kpi: pd.Dataframe # scroll below to see how does it look like
target_rel = '2.0'
comp_rels = ['1.1', '1.2', '1.3']
```

`OUT`:

   rel    | 1.1 | 1.2 | 1.3 
------------ | ------------- | ------------- | ------------- 
PL|	-0.6613|	-0.2882|	-0.2882
DE|	-0.2677|	-0.2677|	-0.2677
EN|	-0.5147|	-0.5147|	-0.1400
IT|	0.0253|	0.0000|	0.0000
ES|	-0.3833|	-0.3833|	-0.3833
PT|	-0.4916|	-0.4916|	0.0000
RU|	0.5926|	0.6805|	0.6805
FR|	-0.3206|	-0.3206|	-0.3206
NL|	0.0452|	0.0452|	0.0452
GR|	-0.3879|	-0.3879|	-0.3879

In [20]:
df_kpi = pd.DataFrame(experiments_data, index=features_index)
df_kpi

,1.1,1.2,1.3,1.4,1.5,2.0,2.1
PL,0.0958,0.4689,0.4689,0.4689,0.4689,0.7571,0.7571
DE,0.0653,0.0653,0.0653,0.3330,0.1910,0.3330,0.3407
EN,0.0414,0.0414,0.4161,0.5561,0.5561,0.5561,0.9218
IT,0.9581,0.9328,0.9328,0.9328,0.9328,0.9328,0.9328
ES,0.4818,0.4818,0.4818,0.4818,0.7878,0.8651,0.8651
PT,0.2347,0.2347,0.7263,0.7263,0.7263,0.7263,0.7263
RU,0.6384,0.7263,0.7263,0.7263,0.7263,0.0458,0.8897
FR,0.2423,0.2423,0.2423,0.5629,0.5629,0.5629,0.5629
NL,0.8497,0.8497,0.8497,0.8497,0.8045,0.8045,0.8897
GR,0.0386,0.0386,0.0386,0.3462,0.4265,0.4265,0.4265


In [21]:
def compare_releases(target_rel, comp_rels, df_kpi):
    ### YOUR CODE HERE:
    raise NotImplementedError
    ### END YOUR CODE
    return result

In [22]:
target_rel = '2.0'
comp_rels = ['1.1', '1.2', '1.3']
# compare_releases(target_rel, comp_rels, df_kpi)

In [23]:
### TO SHOW THE SOLUTION USE LINE BELOW ###
# %load ../91_solutions/ex4_5a.py

#### SOLUTION B: using `pd.merge`/`pd.DataFrame.join` method 

`IN`:
```python
df_tests: pd.Dataframe # scroll below to see how does it look like
target_rel = '2.0'
comp_rels = ['1.1', '1.2', '1.3']
```

`OUT`:

col | release_base|	language|	kpi_base|	release_target|	kpi_target|	difference
----- | ----- | ----- | ----- | ----- | ----- | ----- 
0|	1.1|	PL|	0.0958|	2.0|	0.7571|	-0.6613
1|	1.1|	DE|	0.0653|	2.0|	0.3330|	-0.2677
2|	1.1|	EN|	0.0414|	2.0|	0.5561|	-0.5147
3|	1.1|	IT|	0.9581|	2.0|	0.9328|	0.0253
4|	1.1|	ES|	0.4818|	2.0|	0.8651|	-0.3833


In [24]:
records = []
for release, test_data in experiments_data.items():
    for lang_idx, lang in enumerate(features_index):
        records.append((release, lang, test_data[lang_idx]))
        
df_tests = pd.DataFrame.from_records(records, columns=["release", "language", "kpi"])
df_tests.head()

,release,language,kpi
0,1.1,PL,0.0958
1,1.1,DE,0.0653
2,1.1,EN,0.0414
3,1.1,IT,0.9581
4,1.1,ES,0.4818


In [25]:
def compare_tests(target_rel, comp_rels, df_tests):
    ### YOUR CODE HERE:
    raise NotImplementedError
    ### END YOUR CODE
    return result

In [26]:
target_rel = '2.0'
comp_rels = ['1.1', '1.2', '1.3']
# compare_tests(target_rel, comp_rels, df_tests)

In [27]:
### TO SHOW THE SOLUTION USE LINE BELOW ###
# %load ../91_solutions/ex4_5b.py